In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
import inspect


def simple_coroutine():
    print('-> coroutine started')
    x = yield -1
    print('-> coroutine received:', x)


"""

>>> coro = simple_coroutine()
>>> coro  # doctest: +ELLIPSIS
<generator object simple_coroutine at ...>
>>> inspect.getgeneratorstate(coro)
'GEN_CREATED'
>>> next(coro)
-> coroutine started
-1
>>> inspect.getgeneratorstate(coro)
'GEN_SUSPENDED'
>>> try:
...     coro.send(42)
... except StopIteration:
...     pass
-> coroutine received: 42
>>> inspect.getgeneratorstate(coro)
'GEN_CLOSED'
"""

doctest.testmod()

TestResults(failed=0, attempted=7)

In [4]:
from inspect import getgeneratorstate


def simple_coro2(a):
    print('-> started: a =', a)
    b = yield a
    print('-> received b =', b)
    c = yield a + b
    print('-> received c =', c)


"""

>>> coro = simple_coro2(14)
>>> getgeneratorstate(coro)
'GEN_CREATED'
>>> next(coro)
-> started: a = 14
14
>>> getgeneratorstate(coro)
'GEN_SUSPENDED'
>>> coro.send(28)
-> received b = 28
42
>>> getgeneratorstate(coro)
'GEN_SUSPENDED'
>>> try:
...    d = coro.send(99)
... except StopIteration:
...     pass
-> received c = 99
>>> getgeneratorstate(coro)
'GEN_CLOSED'
"""

doctest.testmod()

TestResults(failed=0, attempted=8)

In [5]:
from functools import wraps


def coroutine(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen

    return wrapper


@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        value = yield average
        total += value
        count += 1
        average = total / count


"""

>>> avg = averager()
>>> avg.send(5)
5.0
>>> avg.send(10)
7.5
>>> avg.send(15)
10.0
>>> avg.send('spam')
Traceback (most recent call last):
    ...
TypeError: unsupported operand type(s) for +=: 'float' and 'str'
>>> avg.send(60)
Traceback (most recent call last):
    ...
StopIteration
"""

doctest.testmod()

TestResults(failed=0, attempted=6)

In [6]:
class DemoException(Exception):
    pass


def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** demo exception')
        else:
            print(f'-> coroutine received: {x!r}')

    raise RuntimeError('this line should never run')


"""

>>> exc_coro = demo_exc_handling()
>>> next(exc_coro)
-> coroutine started
>>> exc_coro.send(1)
-> coroutine received: 1
>>> exc_coro.send('hi')
-> coroutine received: 'hi'
>>> exc_coro.throw(DemoException)
*** demo exception
>>> exc_coro.close()
>>> exc_coro.send(1)
Traceback (most recent call last):
    ...
StopIteration
"""

doctest.testmod()

TestResults(failed=0, attempted=7)

In [7]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

symbol_exit = object()


def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        x = yield
        if x is symbol_exit:
            break

        total += x
        count += 1
        average = total / count

    return Result(count, average)


"""

>>> coro = averager()
>>> next(coro)
>>> coro.send(10)
>>> coro.send(30)
>>> coro.send(6.5)
>>> coro.send(symbol_exit)
Traceback (most recent call last):
    ...
StopIteration: Result(count=3, average=15.5)
"""

doctest.testmod()

TestResults(failed=0, attempted=6)